In [1]:
include("../../BFFPSV18.jl")

┌ Info: Recompiling stale cache file C:\Users\marce\.julia\compiled\v1.2\Reachability\CHV4V.ji for Reachability [774a0091-654f-5c65-bbdc-ad5b67b45832]
└ @ Base loading.jl:1240


add_time! (generic function with 2 methods)

## Model

In [26]:
# ==========================================
# Model: CD-Player (120 variables, 2 inputs)
# ==========================================

using SparseArrays, MAT

# =====================
# Problem specification
# =====================
file = matopen("cdplayer.mat")
A = read(file, "A")

# initial set
X0 = BallInf(zeros(120), 1.0)

# input set
B = read(file, "B")
U = BallInf([0.0, 0.0], 1.0)

# instantiate continuous LTI system
S = ConstrainedLinearControlContinuousSystem(A, B, nothing, U)
P = InitialValueProblem(S, X0)

# prpoperty: 2*x1 -3*x2 < 450.8
property = SafeStatesProperty(HalfSpace(sparsevec([1, 2], [2., -3.], 120), 450.8))

SafeStatesProperty{Float64}(HalfSpace{Float64,SparseVector{Float64,Int64}}(  [1  ]  =  2.0
  [2  ]  =  -3.0, 450.8), Float64[])

## Reach

In [63]:
partition_xD = [(10i+1):10(i+1) for i in 0:5] # 3D blocks
partition_2D = [(2*i-1:2*i) for i in 1:60] # 2D blocks
partition_1D = [[i] for i in 1:120]; # 1D blocks

In [69]:
(10i+1):10(i+1)

UndefVarError: UndefVarError: i not defined

In [64]:
opts = Options(:T => 20.0,  :plot_vars => [0, 1])
opts_algo = Options(:δ => 0.001, :vars => [1], :partition => partition_2D, :assume_sparse => true)

Options(Dict{Symbol,Any}(:vars => [1],:partition => UnitRange{Int64}[1:10, 11:20, 21:30, 31:40, 41:50, 51:60],:assume_sparse => true,:δ => 0.001))

### One variable

In [67]:
sol = solve(P, opts, op=BFFPSV18(opts_algo));

Computing successors 100%|██████████████████████████████| Time: 0:00:01


In [68]:
@btime solve($P, $opts, op=BFFPSV18($opts_algo));

Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01
Computing successors 100%|██████████████████████████████| Time: 0:00:01


  2.197 s (3042475 allocations: 842.82 MiB)


In [ ]:
# 2D, sparse: 940.583 ms (5963491 allocations: 736.43 MiB)
# 2D, dense: 10.543 s (13125383 allocations: 902.78 MiB)

# 1D, sparse:  1.510 s (903552 allocations: 336.36 MiB)
# 1D, dense: 11.270 s (29525662 allocations: 1.51 GiB)

### All variables

### Solution with local implementation

In [ ]:
# UNIFORM 1D partition
opts = Options(:T => 20.0,
               :δ => 0.001,
               :N => 20_000,  # el N se puede sacar de T y de delta
               :vars=>[5],
               :partition => partition_1D,
               :set_type => Interval{Float64, IA.Interval{Float64}},
               :row_blocks => [[5]],
               :column_blocks => partition_1D,
               :num_type => Float64,
               :block_indices => [5]);
# varias de las opciones anteriores se pueden deducir de otras!

In [ ]:
sol = solve_BFFPSV18(P, opts);

In [ ]:
@btime solve_BFFPSV18($P, $opts);

### One variable

### All variables

## Results

## Check

In [ ]:
problem_options = Options(:vars => [1, 2],
                                  :partition => partition,
                                  :property => property,
                                  :assume_sparse => true)